In [ ]:
from modules.db import DatabaseConnection

db = DatabaseConnection()

db.connect()
pedido = db.get_data_from_table('pedido')
representante = db.get_data_from_table('representantes')
meta = db.get_data_from_table('meta2023')


In [ ]:
import pandas as pd

merged_df = pd.merge(pedido, representante, left_on='codrepped_id', right_on='id_repres', how='left')

In [ ]:
merged_df['razsocrep']= merged_df['razsocrep'].replace('WEBER REPRESENTAÇÕES', 'LEANDRO DACOREGIO WEBER')
merged_df

In [ ]:
# Converta a coluna de datas para o tipo datetime se ainda não estiver
merged_df['dataped'] = pd.to_datetime(merged_df['dataped'])

# Crie uma nova coluna 'mes' que contém o mês de cada data
merged_df['mes'] = merged_df['dataped'].dt.month
merged_df['ano'] = merged_df['dataped'].dt.year

# Filtre o DataFrame para incluir apenas registros do ano de 2023
merged_df = merged_df[merged_df['ano'] == 2023]

# Agrupe as informações por mês e representante, somando os valores da coluna 'totalped'
agrupado = merged_df.groupby(['mes', 'razsocrep']).agg({
    'totalped': 'sum'
}).reset_index()

# Exiba as 20 primeiras linhas do DataFrame resultante
print(agrupado.head(20))

In [ ]:
meta.head(5)

In [ ]:
# Converta a coluna de datas para o tipo datetime se ainda não estiver
meta['mês'] = pd.to_datetime(meta['mês'])

# Crie uma nova coluna 'mes' que contém o mês de cada data
meta['mes'] = meta['mês'].dt.month

# Agrupe as informações por mês e representante
agrupado_meta = meta.groupby(['mes', 'representante']).agg({
    'valor': 'sum',  # Substitua 'coluna1' e 'sua_funcao1' pelos nomes reais
}).reset_index()

agrupado_meta.head(5)

In [ ]:
final_df = pd.merge(agrupado, agrupado_meta, left_on=['mes', 'razsocrep'], right_on=['mes', 'representante'], how='outer', suffixes=('_agrupado', '_agrupado_meta'))


In [ ]:
final_df.head(20)

In [ ]:
final_df['rep'] = final_df['razsocrep'].fillna(final_df['representante'])
final_df.rename(columns={'totalped': 'realizado', 'valor':'meta', 'razsocrep': 'excluir', 'representante': 'excluir'}, inplace=True)
final_df.drop('excluir', axis=1, inplace=True)

In [ ]:
final_df.rename(columns={'rep': 'representante'}, inplace=True)
ordem = ['mes', 'representante','realizado', 'meta']
final_df = final_df[ordem]
final_df

In [ ]:
final_df.to_excel('output.xlsx', index=False)